# [ASI Project] Weight Uncertainty in Neural Networks  
**Authors**: Miriam Lamari, Francesco Giannuzzo  


In [41]:
import csv
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
from kaggle_secrets import UserSecretsClient
from torch.utils.data import DataLoader, random_split
import wandb

In [42]:
user_secrets = UserSecretsClient()
key = user_secrets.get_secret('wandb-api-key')

wandb.login(key=key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Minibatches
**minibatch_weight**(batch_idx: int, num_batches: int)

In [43]:
def minibatch_weight(batch_idx: int, num_batches: int) -> float:
    return 2 ** (num_batches - batch_idx) / (2 ** num_batches - batch_idx)

## Variational Approximation

In [44]:
from typing import Any, Optional

import torch.nn as nn
from torch import Tensor


class BayesianModule(nn.Module):

    """Base class for BNN to enable certain behaviour."""

    def __init__(self):
        super().__init__()

    def kld(self, *args):
        raise NotImplementedError('BayesianModule::kld()')


def variational_approximator(model: nn.Module) -> nn.Module:

    def kl_divergence(self) -> Tensor:

        kl = 0
        for module in self.modules():
            if isinstance(module, BayesianModule):
                kl += module.kl_divergence

        return kl

    # add `kl_divergence` to the model
    setattr(model, 'kl_divergence', kl_divergence)

    def elbo(self,
             inputs: Tensor,
             targets: Tensor,
             criterion: Any,
             n_samples: int,
             w_complexity: Optional[float] = 1.0) -> Tensor:

        loss = 0
        for sample in range(n_samples):
            outputs = self(inputs)
            loss += criterion(outputs, targets)
            loss += self.kl_divergence() * w_complexity

        return loss / n_samples

    # add `elbo` to the model
    setattr(model, 'elbo', elbo)

    return model


## Scale Mixture Prior

In [45]:
import functools as ft

import torch
import torch.nn as nn
from torch import Tensor


class ScaleMixture(nn.Module):

    """Scale Mixture Prior.

    Section 3.3 of the 'Weight Uncertainty in Neural Networks' paper
    proposes the use of a Scale Mixture prior for use in variational
    inference - this being a fixed-form prior.

    The authors note that, should the parameters be allowed to adjust
    during training, the prior changes rapidly and attempts to capture
    the empirical distribution of the weights. As a result the prior
    learns to fit poor initial parameters and struggles to improve.
    """

    def __init__(self, pi: float, sigma1: float, sigma2: float) -> None:

        """Scale Mixture Prior.

        The authors of 'Weight Uncertainty in Neural Networks' note:

            sigma1 > sigma2:
                provides a heavier tail in the prior density than is
                seen in a plain Gaussian prior.
            sigma2 << 1.0:
                causes many of the weights to a priori tightly
                concentrate around zero.

        Parameters
        ----------
        pi : float
            Parameter used to scale the two Gaussian distributions.
        sigma1 : float
            Standard deviation of the first normal distribution.
        sigma2 : float
            Standard deviation of the second normal distribution.
        """

        super().__init__()

        self.pi = pi
        self.sigma1 = sigma1
        self.sigma2 = sigma2

        self.normal1 = torch.distributions.Normal(0, sigma1)
        self.normal2 = torch.distributions.Normal(0, sigma2)

    def log_prior(self, w: Tensor) -> Tensor:

        """Log Likelihood of the weight according to the prior.

        Calculates the log likelihood of the supplied weight given the
        prior distribution - the scale mixture of two Gaussians.

        Parameters
        ----------
        w : Tensor
            Weight to be used to calculate the log likelihood.

        Returns
        -------
        Tensor
            Log likelihood of the weights from the prior distribution.
        """

        likelihood_n1 = torch.exp(self.normal1.log_prob(w))
        likelihood_n2 = torch.exp(self.normal2.log_prob(w))

        p_scalemixture = self.pi * likelihood_n1 + (1 - self.pi) * likelihood_n2
        log_prob = torch.log(p_scalemixture).sum()

        return log_prob

## Gaussian Variational Inference

In [46]:
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor


class GaussianVariational(nn.Module):
    def __init__(self, mu: Tensor, rho: Tensor) -> None:

        super().__init__()

        self.mu = nn.Parameter(mu)
        self.rho = nn.Parameter(rho)

        self.w = None
        self.sigma = None

        self.normal = torch.distributions.Normal(0, 1)

    def sample(self) -> Tensor:
        device = self.mu.device
        epsilon = self.normal.sample(self.mu.size()).to(device)
        self.sigma = torch.log(1 + torch.exp(self.rho)).to(device)
        self.w = self.mu + self.sigma * epsilon

        return self.w

    def log_posterior(self) -> Tensor:

        if self.w is None:
            raise ValueError('self.w must have a value.')

        log_const = np.log(np.sqrt(2 * np.pi))
        log_exp = ((self.w - self.mu) ** 2) / (2 * self.sigma ** 2)
        log_posterior = -log_const - torch.log(self.sigma) - log_exp

        return log_posterior.sum()

## Bayesian Linear Layer ##

In [47]:
from typing import Optional

import torch
import torch.nn.functional as F
from torch import Tensor

#from .base_bayesian import BayesianModule
#from .samplers.gaussian_variational import GaussianVariational
#from .samplers.scale_mixture import ScaleMixture


class BayesLinear(BayesianModule):

    """Bayesian Linear Layer.

    Implementation of a Bayesian Linear Layer as described in the
    'Weight Uncertainty in Neural Networks' paper.
    """

    def __init__(self,
                 in_features: int,
                 out_features: int,
                 prior_pi: Optional[float] = 0.5,
                 prior_sigma1: Optional[float] = 1.0,
                 prior_sigma2: Optional[float] = 0.0025) -> None:

        super().__init__()

        w_mu = torch.empty(out_features, in_features).uniform_(-0.2, 0.2)
        w_rho = torch.empty(out_features, in_features).uniform_(-5.0, -4.0)

        bias_mu = torch.empty(out_features).uniform_(-0.2, 0.2)
        bias_rho = torch.empty(out_features).uniform_(-5.0, -4.0)

        self.w_posterior = GaussianVariational(w_mu, w_rho)
        self.bias_posterior = GaussianVariational(bias_mu, bias_rho)

        self.w_prior = ScaleMixture(prior_pi, prior_sigma1, prior_sigma2)
        self.bias_prior = ScaleMixture(prior_pi, prior_sigma1, prior_sigma2)

        self.kl_divergence = 0.0

    def forward(self, x: Tensor) -> Tensor:

        w = self.w_posterior.sample()
        b = self.bias_posterior.sample()

        w_log_prior = self.w_prior.log_prior(w)
        b_log_prior = self.bias_prior.log_prior(b)

        w_log_posterior = self.w_posterior.log_posterior()
        b_log_posterior = self.bias_posterior.log_posterior()

        total_log_prior = w_log_prior + b_log_prior
        total_log_posterior = w_log_posterior + b_log_posterior
        self.kl_divergence = self.kld(total_log_prior, total_log_posterior)

        return F.linear(x, w, b)

    def kld(self, log_prior: Tensor, log_posterior: Tensor) -> Tensor:
        return log_posterior - log_prior

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {}

# define transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])


In [49]:
# load / process data

full_trainset = datasets.MNIST('./data', train=True, download=True, transform=transform)

#train val split

train_size = 50000
val_size = 10000
trainset, valset = random_split(full_trainset, [train_size, val_size])

kwargs = {'shuffle': True, 'num_workers': 2, 'pin_memory': True}
trainloader = DataLoader(trainset, batch_size=128, **kwargs)
valloader = DataLoader(valset, batch_size=128, **kwargs)

testset = datasets.MNIST('./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=128, **kwargs)

In [50]:
def train_loop(learning_rate, prior_pi, prior_sigma1, prior_sigma2):
    
    @variational_approximator
    class BayesianNetwork(nn.Module):
        def __init__(self, input_dim, output_dim):
            super().__init__()
            self.bl1 = BayesLinear(input_dim, 1200, prior_pi, prior_sigma1, prior_sigma2)
            self.bl2 = BayesLinear(1200, 1200, prior_pi, prior_sigma1, prior_sigma2)
            self.bl3 = BayesLinear(1200, output_dim, prior_pi, prior_sigma1, prior_sigma2)
    
        def forward(self, x):
            x = x.view(-1, 28 * 28)
    
            x = F.relu(self.bl1(x))
            x = F.relu(self.bl2(x))
            x = self.bl3(x)
    
            return x
            
    model = BayesianNetwork(28 * 28, 10).to(device)
    
    run = wandb.init(project="Project-ASI")

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(reduction='sum')
    

    min_test_loss = np.Inf
    for epoch in range(1):
    
        train_loss = 0.0
        val_loss = 0.0
    
        model.train()
        for batch_idx, (data, labels) in enumerate(trainloader):
            data, labels = data.to(device), labels.to(device)
    
            optimizer.zero_grad()
    
            pi_weight = minibatch_weight(batch_idx=batch_idx, num_batches=128)
    
            loss = model.elbo(
                inputs=data,
                targets=labels,
                criterion=criterion,
                n_samples=3,
                w_complexity=pi_weight
            )
    
            train_loss += loss.item() * data.size(0)
    
            loss.backward()
            optimizer.step()
    
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, labels) in enumerate(valloader):
                data, labels = data.to(device), labels.to(device)
    
                outputs = model(data)
    
                pi_weight = minibatch_weight(batch_idx=batch_idx, num_batches=128)
    
                loss = model.elbo(
                    inputs=data,
                    targets=labels,
                    criterion=criterion,
                    n_samples=3,
                    w_complexity=pi_weight
                )
    
                val_loss += loss.item() * data.size(0)
    
        train_loss /= len(trainloader.dataset)
        val_loss /= len(valloader.dataset)

        metrics = {'train_loss': train_loss, 'val_loss': val_loss }
        wandb.log(metrics)


In [51]:
def train_wrapper():
    run = wandb.init(project="Project-ASI")

    return train_loop(
        learning_rate = wandb.config.learning_rate,
        prior_pi = wandb.config.prior_pi,
        prior_sigma1=wandb.config.prior_sigma1,
        prior_sigma2=wandb.config.prior_sigma2
    )

## Tuning Hyperparamters

In [52]:
sweep_configuration = {
     "method": "grid",
     "metric": {"goal": "minimize", "name": "val_loss"},
     'name': "sweep-BBB-Gaussian",
     "parameters": {
         "learning_rate": {'values': [1e-3, 1e-4]},
         "prior_pi": {'values': [0.25, 0.5, 0.75]},
         "prior_sigma1": {'values': [1, math.exp(-1), math.exp(-2)]},
         "prior_sigma2": {'values': [math.exp(-6), math.exp(-7), math.exp(-8)]},
     },
}

sweep_configuration = {
     "method": "grid",
     "metric": {"goal": "minimize", "name": "val_loss"},
     'name': "sweep-BBB-test",
     "parameters": {
         "learning_rate": {'values': [1e-3]},
         "prior_pi": {'values': [0.5]},
         "prior_sigma1": {'values': [1]},
         "prior_sigma2": {'values': [math.exp(-6)]},
     },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="Project-ASI")
wandb.agent(sweep_id, function=train_wrapper);

Create sweep with ID: 135kyutr
Sweep URL: https://wandb.ai/miriam-lamari2-eurecom/Project-ASI/sweeps/135kyutr


wandb: Agent Starting Run: bazl5lxe with config:
wandb: 	learning_rate: 0.001
wandb: 	prior_pi: 0.5
wandb: 	prior_sigma1: 1
wandb: 	prior_sigma2: 0.0024787521766663585


train_loss,▁
val_loss,▁
train_loss,55623.21837
val_loss,273039.85421


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
